# **Esercitazione su object detection**
Nell'esercitazione odierna utilizzeremo l'architettura SSD per rilevare gli oggetti nelle immagini del database [PASCAL VOC 2007](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/).

Faremo uso del framework **TensorFlow**, sfruttando la libreria open-source **Keras** appositamente progettata per permettere una rapida prototipazione di reti neurali profonde.

# **Operazioni preliminari**
Eseguendo la cella sottostante tutto il materiale necessario per lo svolgimento dell'esercitazione verrà scaricato sulla macchina remota. Alla fine dell'esecuzione selezionare il tab **Files** per verificare che tutto sia stato scaricato correttamente.

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!wget http://bias.csr.unibo.it/VR/Esercitazioni/MaterialeEsObjectDetection.zip

!tar -xvf /content/VOCtrainval_06-Nov-2007.tar
!tar -xvf /content/VOCtest_06-Nov-2007.tar
!unzip /content/MaterialeEsObjectDetection.zip

!rm /content/VOCtrainval_06-Nov-2007.tar
!rm /content/VOCtest_06-Nov-2007.tar
!rm /content/MaterialeEsObjectDetection.zip

# **Import delle librerie**
Per prima cosa è necessario eseguire l'import delle librerie utilizzate durante l'esecitazione.

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
import keras
import tensorflow as tf
from math import ceil
import numpy as np
from matplotlib import pyplot as plt
import cv2
import random

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from misc.ssd_box_encode_decode_utils import decode_y

%matplotlib inline

# **Dataset per l'addestramento**
In questa esercitazione useremo il dataset della competizione PASCAL VOC 2007.

Il dataset consiste in un insieme di immagini RGB. Ogni immagine ha associato un file di annotazioni in formato XML (*ground truth*) contenente 
la classe e le bounding box relative a tutti gli oggetti presenti nell'immagine.

Eseguendo la cella sottostante il dataset sarà caricato in memoria.

Quando si usano database di grosse dimensioni non sempre è possibile e conveniente caricarli interamente in memoria. Per ovviare a questo problema è possibile utilizzare un *generator* che permette di gestire in maniera efficiente i dati caricandoli in memoria un batch alla volta. Generator più avanziati permettono anche di applicare ai dati tecniche di *data augmentation*. Nel nostro caso utilizzeremo un generator non tanto per questioni di memoria, ma per incrementare le dimensioni e la variabilità del training set tramite *data augmentation*.

In [ ]:
# Percorsi da cui caricare il database
images_dir = '/content/VOCdevkit/VOC2007/JPEGImages/'
annotations_dir = '/content/VOCdevkit/VOC2007/Annotations/'
train_image_set_filename = '/content/VOCdevkit/VOC2007/ImageSets/Main/train.txt'
val_image_set_filename = '/content/VOCdevkit/VOC2007/ImageSets/Main/val.txt'
test_image_set_filename = '/content/VOCdevkit/VOC2007/ImageSets/Main/test.txt'

# Classi degli oggetti presenti nel database
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

# Creazione di due DataGenerator: per il training e il validation set 
train_dataset = DataGenerator(load_images_into_memory=True)
val_dataset = DataGenerator(load_images_into_memory=True)

# Caricamento delle immagini e delle corrispondenti annotazioni di ground truth
train_dataset.parse_xml(images_dirs=[images_dir],
                        image_set_filenames=[train_image_set_filename],
                        annotations_dirs=[annotations_dir],
                        classes=classes)

val_dataset.parse_xml(images_dirs=[images_dir],
                      image_set_filenames=[val_image_set_filename],
                      annotations_dirs=[annotations_dir],
                      classes=classes)

train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print('Numero di immagini del training set:\t{:>6}'.format(train_dataset_size))
print('Numero di immagini del validation set:\t{:>6}'.format(val_dataset_size))

## **Visualizzazione dei dati**
Per capire meglio la natura e la difficoltà del problema che affronteremo può essere molto utile visualizzare alcune immagini di esempio. Eseguendo la cella sottostante verranno visualizzate due immagini del training set scelte casualmente (con le rispettive annotazioni).

In [ ]:
colors = plt.cm.hsv(np.linspace(0, 1, len(classes)+1)).tolist()

_, axs = plt.subplots(1, 2,figsize=(20, 10))
for i in range(2):
  random_idx=random.randint(0,train_dataset_size)
  axs[i].imshow(train_dataset.images[random_idx]),axs[i].axis('off'),axs[i].set_title(train_dataset.image_ids[random_idx])
  for box in train_dataset.labels[random_idx]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    color = colors[int(box[0])]
    label = '{}'.format(classes[int(box[0])])
    axs[i].add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    axs[i].text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})

# **SSD**
Il modello SSD (*Single Shot Detector*) è una rete in grado di individuare in tempo reale oggetti multipli all'interno di un'immagine con un elevato livello di accuratezza.

![alt text](https://miro.medium.com/max/1948/1*51joMGlhxvftTxGtA4lA7Q.png)

## **Iperparametri**
Nella cella sottostante sono riportati gli iperparametri più importanti per la creazione della rete.

In [ ]:
img_height = 300 # Altezza dell'immagine di input del modello
img_width = 300 # Larghezza dell'immagine di input del modello
img_channels = 3 # Numero di canali dell'immagine di input del modello
n_classes = 20 # Numero di classi del problema
scales = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # Fattori di scala delle default box
aspect_ratios = [[1.0, 2.0, 0.5], # Aspect ratio delle default box per ogni predictor layer
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] 
steps = [8, 16, 32, 64, 100, 300] # Distanza tra default box adiacenti per ogni predictor layer
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # Offset del centro della prima default box rispetto al bordo dell'immagine riportato come frazione dello step per ogni prediction layer 

## **Creazione del modello**

Per istanziare un modello della SSD-300, utilizzando gli iperparametri appena impostati, è sufficiente richiamare la funzione **ssd_300(...)**. 

Con il metodo [**load_weights(...)**](https://keras.io/api/models/model_saving_apis/#loadweights-method) della classe **Model** è possibile caricare dei pesi del modello precedentemente salvati. Nel nostro caso verranno caricati i pesi della VGG-16 ottenuti addestrandola sulle immagini di ImageNet.

La classe **SSDLoss** implementa la multi-task loss per SSD.

Infine con il metodo **compile(...)** è possibile configurare *optimizer* e *loss* del modello.


In [ ]:
# File dei pesi iniziali del backbone VGG-16
weights_path = '/content/weights/VGG_ILSVRC_16_layers_fc_reduced.h5'
mean_color = [123, 117, 104] # Valore medio delle immagini utilizzate per addestrare la VGG-16

# Modelli creati in precedenza vengono cancellati
K.clear_session()

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                steps=steps,
                offsets=offsets,
                subtract_mean=mean_color)

# Caricamento dei pesi iniziali della rete
model.load_weights(weights_path, by_name=True)

# Multi-task loss
ssd_loss=SSDLoss()

model.compile(optimizer='Adam', loss=ssd_loss.compute_loss)

### **Visualizzazione del modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura della rete.

In [ ]:
model.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True, show_layer_names=True)

# **Training**
Ora siamo pronti per l'addestramento della SSD utilizzando le immagini del training set per un numero di epoche pari a *epoch_count* utilizzando *minibatch* di dimensione *batch_size*. Durante l'addestramento le prestazioni della rete verranno valutate anche sul validation set.

La classe **SSDDataAugmentation** permette di applicare le trasformazioni di *data aumentation* utilizzate nell'addestramento di SSD.

La classe **SSDInputEncoder** viene utilizzata per trasformare le annotazioni del ground truth (classe e bounding box) nel formato richiesto dal modello SSD.

In [ ]:
# Numero di epoche di addestramento
epoch_count=1

# Numero di pattern all'interno di ogni minibatch
batch_size=32

# Data augmentation per il training set
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# Trasformazioni da applicare al validation set per renderlo conforme alle specifiche della rete
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# Per creare le default box è necessario specificare le dimensioni spaziali dei predictor layer
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                  model.get_layer('fc7_mbox_conf').output_shape[1:3],
                  model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                  model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                  model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                  model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

# Permette di trasformare le annotazionie del ground truth nel formato richiesto per addestrare l'SSD
ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    steps=steps,
                                    offsets=offsets,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5)
                                    
# Creazione dei generator per il training e il validation set da passare al metodo fit
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,resize],
                                     label_encoder=ssd_input_encoder)

Nella cella seguente viene richiamato il metodo [**fit(...)**](https://keras.io/api/models/model_training_apis/#fit-method) che esegue l'intera fase di addestramento in maniera automatica monitorando costantemente la loss function sul training e validation set.

In [ ]:
model.fit(x=train_generator,
          validation_data=val_generator,
          steps_per_epoch=ceil(train_dataset_size)/batch_size,
          validation_steps=ceil(val_dataset_size)/batch_size,
          epochs=epoch_count,
          verbose=1)

# **Caricamento di un modello preaddestrato**
Come avrete notato l'addestramento di una rete di queste dimensioni può richiedere diverse ore.

Pertanto, per proseguire l'esercitazione, creeremo un nuovo modello caricando i pesi dell'intera rete addestrata utilizzando training e validation set di PASCAL VOC 2007 e 2012.

<u>Nota:</u> prima di eseguire la cella assicurarsi che gli iperparametri siano stati definiti nell'apposita sezione.

In [ ]:
# File dei pesi dell'intera SSD-300 addestrata su Pascal VOC 2007+2012 con 120000 iterazioni
weights_path = '/content/weights/VGG_VOC0712_SSD_300x300_iter_120000.h5'
mean_color = [127.5,127.5,127.5] # Valore medio delle immagini utilizzate per addestrare l'intera SSD-300

# Modelli creati in precedenza vengono cancellati
K.clear_session()

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='inference',
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                steps=steps,
                offsets=offsets,
                subtract_mean=mean_color)
                
# Caricamento dei pesi iniziali della rete
model.load_weights(weights_path, by_name=True)

# **Homework (facoltativo)**
Provare a portare a termine l'addestramento utilizzando almeno 100 epoche e salvando il modello (o solamente i pesi ottenuti) con i metodi [**save(...)**](https://keras.io/api/models/model_saving_apis/#save-method) o [**save_weights(...)**](https://keras.io/api/models/model_saving_apis/#saveweights-method) variando i seguenti iperparametri:
- il numero di epoche (*epoch_count*);
- l'ottimizzatore (*optimizer*) utilizzato dalla rete per implementare la *back propagation*. Keras mette a disposizione vari [ottimizzatori](https://keras.io/api/optimizers/#available-optimizers);
- i singoli iperparametri dell'ottimizzatore di cui il più importante è sicuramente il *learning rate*. Per una descrizione dettagliata dei vari parametri di ogni ottimizzatore fare riferimento alla documentazione di Keras.

# **Funzioni per valutare le prestazioni**
Una delle metriche più utilizzate per valutare l'accuratezza di un sistema di *object detection* è la [***medium Average Precision***](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173) (mAP), calcolata come la media su tutte le classi del problema, dell'area sottesa alla curva *Recall/Precision* di ogni singola classe.

Di seguito sono riportate le funzioni utilizzate per calcolare l'mAP:
- **compute_overlap(...)** - calcola l'[*Intersection over Union*](https://en.wikipedia.org/wiki/Jaccard_index) (IoU) tra due bounding box;
- **compute_ap(...)** - calcola l'*Average Precision* a partire dalla curva Recall/Precision;
- **select_preds(...)** - seleziona le regioni più promettenti restituite dalla rete riscalando le bounding box alle dimensioni dell'immagine originale;
- **extract_detections_and_annotations(...)** - i vari output della **detect(...)** vengono raggruppati rispetto alle classi degli oggetti effettivamente presenti nell'immagine (ground truth);
- **compute_true_and_false_positives(...)** - per ogni classe calcola *True Positive* e *False Positive*;
- **compute_class_average_precision(...)** - calcola l'*Average Precision* a partire da *True Positive* e *False Positive* sfruttando la **compute_ap(...)**;
- **compute_medium_average_precision(...)** - calcola la mAP a partire dalle *Average Precision* di tutte le classi.

Eseguire la cella sottostande per definire tutte queste funzioni.

In [ ]:
def compute_overlap(a, b):
    
    #Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    #Parameters
    #----------
    #a: (N, 4) ndarray of float
    #b: (K, 4) ndarray of float
    #Returns
    #-------
    #overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    
    area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

    iw = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(np.expand_dims(a[:, 0], 1), b[:, 0])
    ih = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(np.expand_dims(a[:, 1], 1), b[:, 1])

    iw = np.maximum(iw, 0)
    ih = np.maximum(ih, 0)

    ua = np.expand_dims((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), axis=1) + area - iw * ih

    ua = np.maximum(ua, np.finfo(float).eps)

    intersection = iw * ih

    return intersection / ua


def compute_ap(recall, precision):
    #Compute the average precision, given the recall and precision curves.
    #Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
    #    recall:    The recall curve (list).
    #    precision: The precision curve (list).
    # Returns
    #    The average precision as computed in py-faster-rcnn.
    
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def select_preds(y_pred,output_image_shape,confidence_threshold=0.5):
  y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

  pred_boxes = []
  pred_labels = []
  for box in y_pred_thresh[0]:
    xmin = box[2] * output_image_shape[1] / img_width
    ymin = box[3] * output_image_shape[0] / img_height
    xmax = box[4] * output_image_shape[1] / img_width
    ymax = box[5] * output_image_shape[0] / img_height
    class_id = int(box[0])
    score = box[1]

    pred_boxes.append([xmin, ymin, xmax, ymax, score])
    pred_labels.append(class_id)

  return np.array(pred_boxes),np.array(pred_labels)


def extract_detections_and_annotations(pred_boxes,pred_labels,true_labels):
  detections = [None for i in range(len(classes))]
  annotations = [None for i in range(len(classes))]

  l = range(1, len(classes))
  for label in l:
      if(len(pred_labels)):
          detections[label] = pred_boxes[pred_labels == label, :]

  for label in l:
      if len(true_labels) > 0:
          annotations[label] = true_labels[true_labels[:, 0] == label, 1:5].copy()
      else:
          annotations[label] = np.array([[]])

  return detections,annotations


def compute_true_and_false_positives(all_detections,all_annotations,label,conf_threshold=0.5):
  false_positives = np.zeros((0,))
  true_positives = np.zeros((0,))
  scores = np.zeros((0,))
  num_annotations = 0.0

  for i in range(len(all_detections)):
      annotations = all_annotations[i][label]
      annotations = annotations.astype(np.float32)

      num_annotations += annotations.shape[0]
      detected_annotations = []
      detections = all_detections[i][label]
      if(detections is not None):
          detections = detections.astype(np.float32)

          for d in detections:
              scores = np.append(scores, d[4])

              try:
                  annotations[0][0]
              except IndexError:
                  false_positives = np.append(false_positives, 1)
                  true_positives = np.append(true_positives, 0)
                  continue

              overlaps = compute_overlap(np.expand_dims(d, axis=0), annotations)
              assigned_annotation = np.argmax(overlaps, axis=1)
              max_overlap = overlaps[0, assigned_annotation]
              
              if max_overlap >= conf_threshold and assigned_annotation not in detected_annotations:
              
                  false_positives = np.append(false_positives, 0)
                  true_positives = np.append(true_positives, 1)
                  detected_annotations.append(assigned_annotation)
              else:
                  false_positives = np.append(false_positives, 1)
                  true_positives = np.append(true_positives, 0)

  return true_positives,false_positives,scores,num_annotations


def compute_class_average_precision(true_positives,false_positives,scores,num_annotations):
  if num_annotations == 0:
      return 0
  
  indices = np.argsort(-scores)
  false_positives = false_positives[indices]
  true_positives = true_positives[indices]

  false_positives = np.cumsum(false_positives)
  true_positives = np.cumsum(true_positives)

  recall = true_positives / num_annotations
  
  precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

  return compute_ap(recall, precision)


def compute_medium_average_precision(average_precisions):
  count = 0
  for k in average_precisions.keys():
    count  = count + float(average_precisions[k])

  return count/len(range(1, len(classes)))

# **Esempio di detection sul validation set**
Per eseguire la *detection* utilizzare il metodo **predict(...)** passandogli in input le immagini da analizzare (dopo averle riscalate alle dimensioni di input della rete).

Sfruttando le funzioni definite in precedenza è possibile selezionare le regioni più promettenti.

Il parametro *confidence_threshold* viene utilizzato per selezionare le regioni finali dall'insieme di bounding box restituite dalla rete.


In [ ]:
confidence_threshold = 0.5 # Valore di confidenza utilizzato per la selezione delle regioni
show_gt = False # Flag per la visualizzazione del ground truth
val_image_count = 4 # Numero di immagini da valutare

# Selezione casuale di val_image_count immagini del validation set e loro ridimensionamento
random_indices=[]
input_images=[]
for i in range(val_image_count):
  random_idx=random.randint(0,val_dataset_size)
  resized_image=cv2.resize(val_dataset.images[random_idx],(img_height,img_width))
  random_indices.append(random_idx)
  input_images.append(resized_image)

# Detection
y_preds = model.predict(np.array(input_images))
print('Shape del volume di output: ',y_preds.shape)

all_detections = [None for i in range(val_image_count)]
all_annotations = [None for i in range(val_image_count)]

colors = plt.cm.hsv(np.linspace(0, 1, len(classes)+1)).tolist()
_, axs = plt.subplots(1, val_image_count,figsize=(20, 10))
for i in range(val_image_count):
  # Selezione delle regioni più promettenti
  pred_boxes,pred_labels=select_preds(y_preds[i][np.newaxis,:,:],val_dataset.images[random_indices[i]].shape,confidence_threshold)

  true_labels = np.array(val_dataset.labels[random_indices[i]])
  detections,annotations=extract_detections_and_annotations(pred_boxes,pred_labels,true_labels)

  all_detections[i]=detections
  all_annotations[i]=annotations

  # Visualizzazione dell'immagine
  axs[i].imshow(val_dataset.images[random_indices[i]]),axs[i].axis('off'),axs[i].set_title(val_dataset.image_ids[random_indices[i]])

  # Visualizzazione del ground truth
  if show_gt:
    for box in val_dataset.labels[random_indices[i]]:
      xmin = box[1]
      ymin = box[2]
      xmax = box[3]
      ymax = box[4]
      color = colors[int(box[0])]
      label = '{}'.format(classes[int(box[0])])
      axs[i].add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
      axs[i].text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})

  # Visualizzazione degli oggetti individuati
  for j in range(pred_boxes.shape[0]):
    box=pred_boxes[j]
    label=pred_labels[j]
    xmin = box[0]
    ymin = box[1]
    xmax = box[2]
    ymax = box[3]
    color = colors[int(label)]
    label = '{}: {:.2f}'.format(classes[int(label)], box[4])
    axs[i].add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    axs[i].text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})

## **Valutazione delle prestazioni**
Eseguendo la cella seguente verrà calcolata l'*Average Precision* per ogni classe e l'mAP sui risultati ottenuti nella cella precedente.

In [ ]:
average_precisions = {}

# Calcolo dell'Average Precision per ogni classe del problema
for label in range(1, len(classes)):
  true_positives,false_positives,scores,num_annotations=compute_true_and_false_positives(all_detections,all_annotations,label)
  ap=compute_class_average_precision(true_positives,false_positives,scores,num_annotations)
  average_precisions[classes[label]]=ap

print(average_precisions)
print('mAP: ',compute_medium_average_precision(average_precisions))

# **Valutazione delle prestazioni sul test set**
Misurare le prestazioni sul dataset di test per verificarne l'effettiva capacità di generalizzazione.

## **Test set**
Eseguendo la cella sottostante il dataset di test sarà caricato in memoria.

In [ ]:
# Creazione del DataGenerator
test_dataset = DataGenerator(load_images_into_memory=True)

# Caricamento delle immagini e delle corrispondenti annotazioni di ground truth
test_dataset.parse_xml(images_dirs=[images_dir],
                      image_set_filenames=[test_image_set_filename],
                      annotations_dirs=[annotations_dir],
                      classes=classes)

test_dataset_size = test_dataset.get_dataset_size()

print('Numero di immagini del test set:\t{:>6}'.format(test_dataset_size))

### **Visualizzazione dei dati**
Eseguendo la cella sottostante verranno visualizzate due immagini del test set scelte casualmente (con le rispettive annotazioni).

In [ ]:
colors = plt.cm.hsv(np.linspace(0, 1, len(classes)+1)).tolist()

_, axs = plt.subplots(1, 2,figsize=(20, 10))
for i in range(2):
  random_idx=random.randint(0,test_dataset_size)
  axs[i].imshow(test_dataset.images[random_idx]),axs[i].axis('off'),axs[i].set_title(test_dataset.image_ids[random_idx])
  for box in test_dataset.labels[random_idx]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    color = colors[int(box[0])]
    label = '{}'.format(classes[int(box[0])])
    axs[i].add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    axs[i].text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})

## **Object detection e calcolo delle prestazioni**
Eseguendo la cella sottostante si effettuerà la *detection* sulle immagini del test set utilizzando il modello attuale (*model*).

Per ogni immagine analizzata verrà riportata l'*Average Precision* per ogni classe.

Alla fine dell'esecuzione sarà riportato il valore di mAP.

<u>Nota:</u> visto l'alto numero di immagini contenute nel test set, inizialmente si consiglia di eseguire la *detection* solo su un sottoinsieme (impostando opportunamente la variabile *test_image_count*).

In [ ]:
confidence_threshold=0.5 # Valore di confidenza utilizzato per la selezione delle regioni
test_image_count=200 # Numero di immagini del test set da valutare

all_detections = [None for i in range(test_image_count)]
all_annotations = [None for i in range(test_image_count)]
for i in range(test_image_count):
    print(test_dataset.image_ids[i], end = '')
        
    # Riscalatura dell'immagine alle dimensioni di input della rete
    resized_image=cv2.resize(test_dataset.images[i],(img_height,img_width))

    # Detection
    y_pred = model.predict(resized_image[np.newaxis,:,:,:])

    # Selezione delle regioni più promettenti
    pred_boxes,pred_labels=select_preds(y_pred,test_dataset.images[i].shape,confidence_threshold)

    true_labels = np.array(test_dataset.labels[i])
    detections,annotations=extract_detections_and_annotations(pred_boxes,pred_labels,true_labels)
    
    all_detections[i]=detections
    all_annotations[i]=annotations

    true_label_aps={}
    for label in true_labels:
      true_positives,false_positives,scores,num_annotations=compute_true_and_false_positives([detections],[annotations],label[0])
      ap=compute_class_average_precision(true_positives,false_positives,scores,num_annotations)
      true_label_aps[classes[label[0]]]=ap

    print('\t',true_label_aps)

# Calcolo dell'Average Precision per ogni classe del problema
average_precisions = {}
for label in range(1, len(classes)):
  true_positives,false_positives,scores,num_annotations=compute_true_and_false_positives(all_detections,all_annotations,label)
  ap=compute_class_average_precision(true_positives,false_positives,scores,num_annotations)
  average_precisions[classes[label]]=ap

print(average_precisions)
print('mAP: ',compute_medium_average_precision(average_precisions))

### **Selezione della soglia di confidenza ottimale**
Verificare l'influenza della soglia di confidenza sulle prestazioni della rete variando il parametro *confidence_threshold* e rieseguendo la cella precedente. 

## **Visualizzare il risultato su singola immagine**
Eseguendo la cella sottostante sarà possibile visualizzare il risultato della *detection* su una specifica immagine del test set (impostando opportunamente la variabile *image_id*).

In [ ]:
confidence_threshold = 0.5 # Valore di confidenza utilizzato per la selezione delle regioni
show_gt = True # Flag per la visualizzazione del ground truth
image_id ='000001' # ID dell'immagine del test set da processare

image_idx=test_dataset.image_ids.index(image_id)

# Riscalatura dell'immagine alle dimensioni di input della rete
resized_image=cv2.resize(test_dataset.images[image_idx],(img_height,img_width))

# Detection
y_pred = model.predict(resized_image[np.newaxis,:,:,:])

# Selezione delle regioni più promettenti
pred_boxes,pred_labels=select_preds(y_pred,test_dataset.images[image_idx].shape,confidence_threshold)

# Visualizzazione dell'immagine
_, ax = plt.subplots(figsize=(10,10))
ax.imshow(test_dataset.images[image_idx]),ax.axis('off')

# Visualizzazione del ground truth
if show_gt:
  for box in test_dataset.labels[image_idx]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    color = colors[int(box[0])]
    label = '{}'.format(classes[int(box[0])])
    ax.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
    ax.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})

# Visualizzazione degli oggetti individuati
for j in range(pred_boxes.shape[0]):
  box=pred_boxes[j]
  label=pred_labels[j]
  xmin = box[0]
  ymin = box[1]
  xmax = box[2]
  ymax = box[3]
  color = colors[int(label)]
  label = '{}: {:.2f}'.format(classes[int(label)], box[4])
  ax.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
  ax.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})